# Model 3 : Using Datetime Features to predict Energy Load

In [1]:
import os
import numpy as np 
import pandas as pd 
import plotly.graph_objs as go 
import plotly.io as pio
import plotly.express as px

import torch 
import torch.nn as nn 
import torch.optim as optim

pio.templates.default = "plotly_white"

from termcolor import colored

from datetime import datetime
import holidays

from utils import load_data

print(f"cwd : {os.getcwd()}")

cwd : /Users/imantha/workspace/cryo-polygen/ts-forecasting/model3


## Load Data

In [2]:
path = os.path.join("..", "data", "load.xlsx")
df = load_data(path, last_date = "2022-05-15 23:00:00", endpoint=31)
df.rename(columns = {"y" : "value", "date" : "Datetime"}, inplace = True)
df.set_index("Datetime", inplace = True)
df.head()

,value
Datetime,
2021-11-15 19:00:00,447
2021-11-15 20:00:00,435
2021-11-15 21:00:00,451
2021-11-15 22:00:00,442
2021-11-15 23:00:00,444


In [3]:
p = go.Figure()
p.add_trace(go.Scatter(
    x = df.index,
    y = df.value
))

## Lagged Observations as features

In [4]:
def generate_time_lags(df, n_lags):
    df_n = df.copy()
    for n in range(1, n_lags + 1):
        df_n[f"lag{n}"] = df_n["value"].shift(n)
    df_n = df_n.iloc[n_lags:]
    return df_n

df_generated = generate_time_lags(df, 24)
df_generated

,value,lag1,lag2,lag3,lag4,lag5,lag6,lag7,lag8,lag9,...,lag15,lag16,lag17,lag18,lag19,lag20,lag21,lag22,lag23,lag24
Datetime,,,,,,,,,,,,,,,,,,,,,
2021-11-16 19:00:00,488,484.0,439.0,436.0,436.0,446.0,498.0,472.0,482.0,510.0,...,459.0,480.0,476.0,447.0,446.0,444.0,442.0,451.0,435.0,447.0
2021-11-16 20:00:00,425,488.0,484.0,439.0,436.0,436.0,446.0,498.0,472.0,482.0,...,500.0,459.0,480.0,476.0,447.0,446.0,444.0,442.0,451.0,435.0
2021-11-16 21:00:00,432,425.0,488.0,484.0,439.0,436.0,436.0,446.0,498.0,472.0,...,468.0,500.0,459.0,480.0,476.0,447.0,446.0,444.0,442.0,451.0
2021-11-16 22:00:00,422,432.0,425.0,488.0,484.0,439.0,436.0,436.0,446.0,498.0,...,447.0,468.0,500.0,459.0,480.0,476.0,447.0,446.0,444.0,442.0
2021-11-16 23:00:00,417,422.0,432.0,425.0,488.0,484.0,439.0,436.0,436.0,446.0,...,468.0,447.0,468.0,500.0,459.0,480.0,476.0,447.0,446.0,444.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-15 19:00:00,517,496.0,517.0,529.0,514.0,563.0,567.0,580.0,583.0,579.0,...,559.0,548.0,561.0,496.0,488.0,501.0,500.0,481.0,503.0,511.0
2022-05-15 20:00:00,514,517.0,496.0,517.0,529.0,514.0,563.0,567.0,580.0,583.0,...,548.0,559.0,548.0,561.0,496.0,488.0,501.0,500.0,481.0,503.0
2022-05-15 21:00:00,511,514.0,517.0,496.0,517.0,529.0,514.0,563.0,567.0,580.0,...,564.0,548.0,559.0,548.0,561.0,496.0,488.0,501.0,500.0,481.0


## Generating Features from TimeStamps

In [5]:
df_features = (
    df
    .assign(hour = df.index.hour)
    .assign(day = df.index.day)
    .assign(month = df.index.month)
    .assign(day_of_week = df.index.dayofweek)
    .assign(week_of_year = df.index.week)
)

/var/folders/_1/j0f_5zq95qd_2dpzlmfh8njm0000gn/T/ipykernel_7687/131758714.py:7: FutureWarning:

weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)



## One-hot encoding

In [6]:
df_features = pd.get_dummies(df_features, columns = ["month", "day", "day_of_week", "week_of_year"])

## Cyclic Features

In [7]:
def generate_cyclic_features(df, col_name, period, start_num = 0):
    kwargs = {
        f"sin_{col_name}" : lambda x: np.sin(2 * np.pi*(df[col_name] - start_num)/period),
        f"cos_{col_name}" : lambda x: np.cos(2 * np.pi*(df[col_name] - start_num)/period)
    }
    return df.assign(**kwargs).drop(columns = [col_name])

df_features = generate_cyclic_features(df_features, "hour", 24, 0)


In [8]:
df_features.head()

,value,month_1,month_2,month_3,month_4,month_5,month_11,month_12,day_1,day_2,...,week_of_year_19,week_of_year_46,week_of_year_47,week_of_year_48,week_of_year_49,week_of_year_50,week_of_year_51,week_of_year_52,sin_hour,cos_hour
Datetime,,,,,,,,,,,,,,,,,,,,,
2021-11-15 19:00:00,447,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,-0.965926,0.258819
2021-11-15 20:00:00,435,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,-0.866025,0.500000
2021-11-15 21:00:00,451,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,-0.707107,0.707107
2021-11-15 22:00:00,442,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,-0.500000,0.866025
2021-11-15 23:00:00,444,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,-0.258819,0.965926


## Holidays

In [9]:
us_holidays = holidays.Singapore()

def is_holiday(date):
    date = date.replace(hour = 0)
    return 1 if (date in us_holidays) else 0

def add_holiday_col(df, holidays):
    return df.assign(is_holiday = df.index.to_series().apply(is_holiday))

df_features = add_holiday_col(df_features, us_holidays)
df_features

,value,month_1,month_2,month_3,month_4,month_5,month_11,month_12,day_1,day_2,...,week_of_year_46,week_of_year_47,week_of_year_48,week_of_year_49,week_of_year_50,week_of_year_51,week_of_year_52,sin_hour,cos_hour,is_holiday
Datetime,,,,,,,,,,,,,,,,,,,,,
2021-11-15 19:00:00,447,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,-0.965926,0.258819,0
2021-11-15 20:00:00,435,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,-0.866025,0.500000,0
2021-11-15 21:00:00,451,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,-0.707107,0.707107,0
2021-11-15 22:00:00,442,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,-0.500000,0.866025,0
2021-11-15 23:00:00,444,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,-0.258819,0.965926,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-15 19:00:00,517,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,-0.965926,0.258819,1
2022-05-15 20:00:00,514,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,-0.866025,0.500000,1
2022-05-15 21:00:00,511,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,-0.707107,0.707107,1


In [10]:
df_generated.drop(columns = "value", inplace = True)
df_features = pd.merge(df_features,df_generated, left_index = True, right_index = True)
df_features

,value,month_1,month_2,month_3,month_4,month_5,month_11,month_12,day_1,day_2,...,lag15,lag16,lag17,lag18,lag19,lag20,lag21,lag22,lag23,lag24
Datetime,,,,,,,,,,,,,,,,,,,,,
2021-11-16 19:00:00,488,0,0,0,0,0,1,0,0,0,...,459.0,480.0,476.0,447.0,446.0,444.0,442.0,451.0,435.0,447.0
2021-11-16 20:00:00,425,0,0,0,0,0,1,0,0,0,...,500.0,459.0,480.0,476.0,447.0,446.0,444.0,442.0,451.0,435.0
2021-11-16 21:00:00,432,0,0,0,0,0,1,0,0,0,...,468.0,500.0,459.0,480.0,476.0,447.0,446.0,444.0,442.0,451.0
2021-11-16 22:00:00,422,0,0,0,0,0,1,0,0,0,...,447.0,468.0,500.0,459.0,480.0,476.0,447.0,446.0,444.0,442.0
2021-11-16 23:00:00,417,0,0,0,0,0,1,0,0,0,...,468.0,447.0,468.0,500.0,459.0,480.0,476.0,447.0,446.0,444.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-15 19:00:00,517,0,0,0,0,1,0,0,0,0,...,559.0,548.0,561.0,496.0,488.0,501.0,500.0,481.0,503.0,511.0
2022-05-15 20:00:00,514,0,0,0,0,1,0,0,0,0,...,548.0,559.0,548.0,561.0,496.0,488.0,501.0,500.0,481.0,503.0
2022-05-15 21:00:00,511,0,0,0,0,1,0,0,0,0,...,564.0,548.0,559.0,548.0,561.0,496.0,488.0,501.0,500.0,481.0


In [11]:
df_features.dtypes

value        int64
month_1      uint8
month_2      uint8
month_3      uint8
month_4      uint8
            ...   
lag20      float64
lag21      float64
lag22      float64
lag23      float64
lag24      float64
Length: 99, dtype: object

## Splitting the data into train, validation and test sets

In [12]:
from sklearn.model_selection import train_test_split

In [46]:
def feature_label_split(df, target_col):
    y = df[[target_col]]
    X = df.drop(columns = [target_col])
    return X, y 

def train_val_test_split(df, target_col, test_ratio):
    val_ratio = test_ratio / (1 - test_ratio)
    X, y = feature_label_split(df, target_col)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, shuffle = False)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio, shuffle = False)
    return X_train, X_val, X_test, y_train, y_val, y_test 

X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(df_features, "value", 0.1)

print("shapes")
print(f"X_train : {X_train.shape}")
print(f"y_train : {y_train.shape}")
print(f"X_val : {X_val.shape}")
print(f"y_val : {y_val.shape}")
print(f"X_test : {X_test.shape}")
print(f"y_test : {y_test.shape}")

shapes
X_train : (3459, 98)
y_train : (3459, 1)
X_val : (433, 98)
y_val : (433, 1)
X_test : (433, 98)
y_test : (433, 1)


## Normalising Data

In [47]:
from sklearn.preprocessing import MinMaxScaler

In [48]:
normaliser = MinMaxScaler()
X_train_norm = normaliser.fit_transform(X_train)
X_val_norm = normaliser.fit_transform(X_val)
X_test_norm = normaliser.fit_transform(X_test)

y_train_norm = normaliser.fit_transform(y_train)
y_val_norm = normaliser.fit_transform(y_val)
y_test_norm = normaliser.fit_transform(y_test)

## Data Loader

In [49]:
from torch.utils.data import TensorDataset, DataLoader

In [50]:
batch_size = 64

train_features = torch.Tensor(X_train_norm)
train_targets = torch.Tensor(y_train_norm)
val_features = torch.Tensor(X_val_norm)
val_targets = torch.Tensor(y_val_norm)
test_features = torch.Tensor(X_test_norm)
test_targets = torch.Tensor(y_test_norm)

train_data = TensorDataset(train_features, train_targets)
val_data = TensorDataset(val_features, val_targets)
test_data = TensorDataset(test_features, test_targets)

train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = False, drop_last = True)
val_loader = DataLoader(val_data, batch_size = batch_size, shuffle = False, drop_last = True)
test_loader = DataLoader(test_data, batch_size = batch_size, shuffle = False, drop_last = True)


## Vanilla RNN

In [51]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, layer_size, output_size, dropout_prob):
        super(RNNModel, self).__init__()

        self.hidden_size = hidden_size
        self.layer_size = layer_size 
        # RNN Layer
        # (N,L,H_in), (numLayers,N, h_out) --> (N,L,H_out), (numlayers, n, hout)
        self.rnn = nn.RNN(input_size = input_size, hidden_size = hidden_size, batch_first = True)
        # Fully connected layers
        # (*, H_in) --> (*,H_out)
        self.fc = nn.Linear(in_features = hidden_size, out_features = output_size)

    def forward(self, x):
        # initialize hidden state
        #(numlayers. N, H_out)
        h0 = torch.zeros(self.layer_size,x.shape[0], self.hidden_size).requires_grad_()
        # Forward pass
        out, hn = self.rnn(x, h0.detach())
        out = out[:, -1, :] 
        out = self.fc(out)
        return out
        

## GRU

In [52]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, layer_size, output_size, dropout_prob):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size 
        self.layer_size = layer_size 

        # GRU Layer
        # (N, L, H_in), (num_layer, N, H_out) ---> (N, L, H_out), (num_layer, N, H_out)
        self.gru = nn.GRU(input_size = input_size, hidden_size = hidden_size, num_layers = layer_size, batch_first = True, dropout = dropout_prob)
        # FCC
        # (*, H_in) ---> (*, H_out)
        self.fc = nn.Linear(in_features=hidden_size, out_features = 1)
        
    def forward(self, x):
        # initialize hidden state
        #(numlayers. N, H_out)
        h0 = torch.zeros(self.layer_size,x.shape[0], self.hidden_size).requires_grad_()
        # Forward pass
        out, hn = self.gru(x, h0.detach())
        out = out[:, -1, :] 
        out = self.fc(out)
        return out


## LSTM

In [53]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, layer_size, output_size, dropout_prob):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size 
        self.layer_size = layer_size 

        # LSTM Layer
        # (N, L, H_in), (num_layer, N, H_out), (num_layer, N, H_cell) ---> (N, L, H_out), (num_layer, N, H_out), (num_layer, N, H_cell)
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = layer_size, batch_first = True, dropout = dropout_prob)
        # FCC
        # (*, H_in) ---> (*, H_out)
        self.fc = nn.Linear(in_features=hidden_size, out_features = 1)
        
    def forward(self, x):
        # initialize hidden state
        #(numlayers, N, H_out)
        h0 = torch.zeros(self.layer_size,x.shape[0], self.hidden_size).requires_grad_()
        c0 = torch.zeros(self.layer_size,x.shape[0], self.hidden_size).requires_grad_()
        # Forward pass
        out, (hn,cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = out[:, -1, :] 
        out = self.fc(out)
        return out

In [54]:
def get_model(model,  model_params):
    models = {
        "rnn" : RNNModel,
        "gru" : GRUModel,
        "lstm" : LSTMModel
    }
    return models.get(model.lower())(**model_params)

## Training

In [55]:
# Training
class Optimizer():
    def __init__(self, model, criterion, optimizer):
        self.model = model 
        self.criterion = criterion 
        self.optimizer = optimizer
        self.training_losses = []
        self.val_losses = []

    def train_step(self, X, y):
        # set the model to train mode
        self.model.train() # allows the weights of the network to be updated
        # Make predictions
        yhat = self.model.forward(X)
        # Compute loss
        loss = self.criterion(y, yhat)
        # Compute gradients
        loss.backward()
        self.optimizer.step()
        self.optimizer.zero_grad()
        # Return losses
        return loss.item()

    def train(self, train_loader, val_loader, batch_size = 64, n_epochs = 50, n_features = 1):
        for epoch in range(1, n_epochs + 1):
            batch_losses = []
            for X_batch, y_batch in train_loader:
                X_batch = X_batch.view([batch_size,  -1, n_features]) # add a dimension at 2, (64, 106) --> (64,106,1)
                loss = self.train_step(X_batch,  y_batch) # Calls the training step herem which calls model.forward()
                batch_losses.append(loss)
            training_loss = np.mean(batch_losses)
            self.training_losses.append(training_loss)

            with torch.no_grad():
                batch_val_losses = []
                for x_val, y_val in val_loader:
                    x_val = x_val.view([batch_size, -1, n_features]) # adds a dimension ar 2, (64, 106) --> 
                    self.model.eval()
                    yhat = model.forward(x_val) # Calls model.forward() method here
                    val_loss = self.criterion(y_val, yhat)
                    batch_val_losses.append(val_loss.item())
                validation_loss = np.mean(batch_val_losses)
                self.val_losses.append(validation_loss)

            if (epoch <= 10) | (epoch%50 == 0):
                print(f"[{epoch}/{n_epochs}] Training loss : {training_loss:.4f}\t Validation loss : {validation_loss:.4f}")

        torch.save(self.model.state_dict(),f"state_dict/example_model_{datetime.now()}")

    def evaluate(self, test_loader, batch_size = 1, n_features = 1):
        with torch.no_grad():
            predictions = []
            values = []
            for x_test, y_test in test_loader:
                x_test = x_test.view([batch_size, -1, n_features]) # (1, 106) --> (1, 106, 6)
                self.model.eval()
                yhat = self.model(x_test)
                predictions.append(yhat.detach().numpy())
                values.append(y_test.detach().numpy())

        return predictions, values

## Inverse Transform

In [56]:
def inverse_transform(scaler,df, columns):
    for col in columns:
        df[col] = scaler.inverse_transform(df[col])
    return df



## Prediction

In [57]:
def format_predictions(predictions, values, df_test, scaler):
    vals = np.concatenate(values, axis = 0).ravel()
    preds = np.concatenate(predictions, axis = 0).ravel()
    df_result = pd.DataFrame(data = {"value" : vals, "predictions" : preds}, index = df_test.head(len(vals)).index)
    df_result = inverse_transform(scaler, df_result, [["value", "predictions"]])
    return df_result

## Metrics

In [58]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def calculate_metrics(df):
    return {'mae' : mean_absolute_error(df.value, df.predictions),
            'rmse' : mean_squared_error(df.value, df.predictions) ** 0.5,
            'r2' : r2_score(df.value, df.predictions)}

# --------------------------------------------------------------------------------------------------

## RNN

In [75]:
input_size = len(X_train.columns) # 106
output_size = 1
hidden_size = 64
layer_size = 1
batch_size = 64
dropout_prob = 0.2 
n_epochs = 500
learning_rate = 1e-3
weight_decay = 1e-6

model_params = {
    "input_size" : input_size,
    "hidden_size" : hidden_size,
    "layer_size" : layer_size, 
    "output_size" : output_size,
    "dropout_prob" : dropout_prob
}

model = get_model("rnn", model_params)
criterion = nn.MSELoss(reduction = "mean")
optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay=weight_decay)
opt = Optimizer(model = model, criterion = criterion, optimizer = optimizer)
opt.train(train_loader=train_loader, val_loader=val_loader, batch_size=batch_size, n_epochs=n_epochs, n_features=input_size)

[1/500] Training loss : 0.0393	 Validation loss : 0.0126
[2/500] Training loss : 0.0157	 Validation loss : 0.0107
[3/500] Training loss : 0.0213	 Validation loss : 0.0103
[4/500] Training loss : 0.0402	 Validation loss : 0.1156
[5/500] Training loss : 0.0293	 Validation loss : 0.0182
[6/500] Training loss : 0.0114	 Validation loss : 0.0124
[7/500] Training loss : 0.0086	 Validation loss : 0.0109
[8/500] Training loss : 0.0081	 Validation loss : 0.0092
[9/500] Training loss : 0.0080	 Validation loss : 0.0112
[10/500] Training loss : 0.0080	 Validation loss : 0.0144
[50/500] Training loss : 0.0095	 Validation loss : 0.0115
[100/500] Training loss : 0.0053	 Validation loss : 0.0102
[150/500] Training loss : 0.0037	 Validation loss : 0.0125
[200/500] Training loss : 0.0042	 Validation loss : 0.0118
[250/500] Training loss : 0.0035	 Validation loss : 0.0176
[300/500] Training loss : 0.0027	 Validation loss : 0.0204
[350/500] Training loss : 0.0033	 Validation loss : 0.0179
[400/500] Trainin

In [76]:
predictions, values = opt.evaluate(test_loader, batch_size = 64, n_features = input_size)

In [77]:
training_losses = opt.training_losses
validation_losses = opt.val_losses

p1 = go.Figure()
p1.add_trace(go.Scatter(y = training_losses, name = "training losses"))
p1.add_trace(go.Scatter(y = validation_losses, name = "validation losses"))
p1.update_layout(xaxis_title = "epochs", yaxis_title = "losses")

In [78]:


df_result = format_predictions(predictions, values, X_test, normaliser)
df_result

,value,predictions
Datetime,,
2022-04-27 23:00:00,497.000000,507.827545
2022-04-28 00:00:00,523.000000,504.921783
2022-04-28 01:00:00,502.000031,530.970032
2022-04-28 02:00:00,593.000000,567.619263
2022-04-28 03:00:00,691.000000,683.291565
...,...,...
2022-05-13 18:00:00,491.000000,504.825775
2022-05-13 19:00:00,510.000031,493.953705
2022-05-13 20:00:00,518.000000,495.311279


In [79]:
result_metrics = calculate_metrics(df_result)
result_metrics

{'mae': 24.819473, 'rmse': 31.76363818798942, 'r2': 0.9289106023536273}

In [80]:
p2 = go.Figure()
p2.add_trace(go.Scatter(x = df_result.index, y = df_result.value))
p2.add_trace(go.Scatter(x = df_result.index, y = df_result.predictions))

## GRU Prediction

In [81]:
input_size = len(X_train.columns) # 106
output_size = 1
hidden_size = 64
layer_size = 1
batch_size = 64
dropout_prob = 0.2 
n_epochs = 500
learning_rate = 1e-3
weight_decay = 1e-6

model_params = {
    "input_size" : input_size,
    "hidden_size" : hidden_size,
    "layer_size" : layer_size, 
    "output_size" : output_size,
    "dropout_prob" : dropout_prob
}

model = get_model("gru", model_params)
criterion = nn.MSELoss(reduction = "mean")
optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay=weight_decay)
opt = Optimizer(model = model, criterion = criterion, optimizer = optimizer)
opt.train(train_loader=train_loader, val_loader=val_loader, batch_size=batch_size, n_epochs=n_epochs, n_features=input_size)

/Users/imantha/Software/miniforge3/envs/mlEnv/lib/python3.8/site-packages/torch/nn/modules/rnn.py:60: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1



[1/500] Training loss : 0.0690	 Validation loss : 0.0141
[2/500] Training loss : 0.0156	 Validation loss : 0.0094
[3/500] Training loss : 0.0105	 Validation loss : 0.0102
[4/500] Training loss : 0.0095	 Validation loss : 0.0216
[5/500] Training loss : 0.0098	 Validation loss : 0.0176
[6/500] Training loss : 0.0104	 Validation loss : 0.0082
[7/500] Training loss : 0.0116	 Validation loss : 0.0085
[8/500] Training loss : 0.0131	 Validation loss : 0.0094
[9/500] Training loss : 0.0126	 Validation loss : 0.0270
[10/500] Training loss : 0.0118	 Validation loss : 0.0792
[50/500] Training loss : 0.0056	 Validation loss : 0.0223
[100/500] Training loss : 0.0037	 Validation loss : 0.0174
[150/500] Training loss : 0.0028	 Validation loss : 0.0141
[200/500] Training loss : 0.0027	 Validation loss : 0.0320
[250/500] Training loss : 0.0022	 Validation loss : 0.0291
[300/500] Training loss : 0.0021	 Validation loss : 0.0316
[350/500] Training loss : 0.0021	 Validation loss : 0.0286
[400/500] Trainin

In [82]:
training_losses = opt.training_losses
validation_losses = opt.val_losses

p1 = go.Figure()
p1.add_trace(go.Scatter(y = training_losses, name = "training losses"))
p1.add_trace(go.Scatter(y = validation_losses, name = "validation losses"))
p1.update_layout(xaxis_title = "epochs", yaxis_title = "losses")

In [83]:
predictions, values = opt.evaluate(test_loader, batch_size = 64, n_features = input_size)

In [84]:
df_result = format_predictions(predictions, values, X_test, normaliser)
df_result

,value,predictions
Datetime,,
2022-04-27 23:00:00,497.000000,495.730927
2022-04-28 00:00:00,523.000000,529.231201
2022-04-28 01:00:00,502.000031,534.898865
2022-04-28 02:00:00,593.000000,558.057312
2022-04-28 03:00:00,691.000000,663.652466
...,...,...
2022-05-13 18:00:00,491.000000,493.981842
2022-05-13 19:00:00,510.000031,486.003235
2022-05-13 20:00:00,518.000000,483.510559


In [85]:
result_metrics = calculate_metrics(df_result)
result_metrics

{'mae': 27.412384, 'rmse': 34.63422323288502, 'r2': 0.9154808093941795}

In [86]:
p2 = go.Figure()
p2.add_trace(go.Scatter(x = df_result.index, y = df_result.value))
p2.add_trace(go.Scatter(x = df_result.index, y = df_result.predictions))

## LSTM Prediction

In [93]:
input_size = len(X_train.columns) # 106
output_size = 1
hidden_size = 64
layer_size = 2
batch_size = 64
dropout_prob = 0.2 
n_epochs = 500
learning_rate = 1e-3
weight_decay = 1e-6

model_params = {
    "input_size" : input_size,
    "hidden_size" : hidden_size,
    "layer_size" : layer_size, 
    "output_size" : output_size,
    "dropout_prob" : dropout_prob
}

model = get_model("lstm", model_params)
criterion = nn.MSELoss(reduction = "mean")
optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay=weight_decay)
opt = Optimizer(model = model, criterion = criterion, optimizer = optimizer)
opt.train(train_loader=train_loader, val_loader=val_loader, batch_size=batch_size, n_epochs=n_epochs, n_features=input_size)

[1/500] Training loss : 0.0439	 Validation loss : 0.0136
[2/500] Training loss : 0.0131	 Validation loss : 0.0120
[3/500] Training loss : 0.0085	 Validation loss : 0.0128
[4/500] Training loss : 0.0074	 Validation loss : 0.0148
[5/500] Training loss : 0.0073	 Validation loss : 0.0268
[6/500] Training loss : 0.0096	 Validation loss : 0.0397
[7/500] Training loss : 0.0137	 Validation loss : 0.0143
[8/500] Training loss : 0.0117	 Validation loss : 0.0148
[9/500] Training loss : 0.0078	 Validation loss : 0.0217
[10/500] Training loss : 0.0054	 Validation loss : 0.0237
[50/500] Training loss : 0.0034	 Validation loss : 0.0192
[100/500] Training loss : 0.0032	 Validation loss : 0.0374
[150/500] Training loss : 0.0029	 Validation loss : 0.0404
[200/500] Training loss : 0.0024	 Validation loss : 0.0285
[250/500] Training loss : 0.0022	 Validation loss : 0.0298
[300/500] Training loss : 0.0020	 Validation loss : 0.0213
[350/500] Training loss : 0.0021	 Validation loss : 0.0224
[400/500] Trainin

In [94]:
training_losses = opt.training_losses
validation_losses = opt.val_losses

p1 = go.Figure()
p1.add_trace(go.Scatter(y = training_losses, name = "training losses"))
p1.add_trace(go.Scatter(y = validation_losses, name = "validation losses"))
p1.update_layout(xaxis_title = "epochs", yaxis_title = "losses")

In [89]:
predictions, values = opt.evaluate(test_loader, batch_size = 64, n_features = input_size)

In [90]:
df_result = format_predictions(predictions, values, X_test, normaliser)
df_result

,value,predictions
Datetime,,
2022-04-27 23:00:00,497.000000,507.233215
2022-04-28 00:00:00,523.000000,508.852417
2022-04-28 01:00:00,502.000031,539.845032
2022-04-28 02:00:00,593.000000,578.526245
2022-04-28 03:00:00,691.000000,679.987671
...,...,...
2022-05-13 18:00:00,491.000000,507.408417
2022-05-13 19:00:00,510.000031,497.435913
2022-05-13 20:00:00,518.000000,494.667023


In [91]:
result_metrics = calculate_metrics(df_result)
result_metrics

{'mae': 28.506205, 'rmse': 35.1290102804114, 'r2': 0.9130486709590778}

In [92]:
p2 = go.Figure()
p2.add_trace(go.Scatter(x = df_result.index, y = df_result.value))
p2.add_trace(go.Scatter(x = df_result.index, y = df_result.predictions))